# Explanation and actual causality

**Preceding notebooks**

- [Actual Causality: the modified Halpern-Pearl definition]() TODO add link

- [Responsibility and actual causality]() TODO add link

- [Blame and actual causality]() TODO add link

**Summary**

Following *Actual Causality* (J. Halpern, MIT Press, 2016), we use the Halpern-Pearl modified definition of actual causality to explicate the notion of an explanation (discussed in ch. 7 of the book).
We will re-use our implementation of actual causality, discussed in a previous notebook.

**Outline**

[Intuitions](##intuitions)
    
[Formalization](#formalization)

[Implementation](#implementation)

[Examples](#examples)

- [Comments on example selection](#comments-on-example-selection)
  
- [Forest fire](#forest-fire)

- [Extended forest fire](#extended-forest-fire)

## Intuitions

On this approach, an explanation is a fact that, if found to be true, would constitute, roughly speaking, an actual cause of the fact to be explained. Moreover, as agents may have different epistemic states, explanations are agent-relative, telling the agents something that they don't already know.

## Formalization

The agent starts with some uncertainty, represented as a set $K$ of causal settings not excluded by what they know (for the sake of simplicity, we consider only causal settings differing in the values of the exogenous variables in the examples).  Relative to $K$, $\vec{X} = \vec{x}$ is an explanation of $\varphi$ iff the following hold:

1. $\vec{X} = \vec{x}$ is a sufficient cause of $\varphi$ in all contexts in $K$ that satisfy $\vec{X} = \vec{x} \wedge \varphi$. That is:

    A. For any $\langle M, \vec{u}\rangle \in K$, if $\langle M, \vec{u}\rangle \models \vec{X} = \vec{x} \wedge \varphi$, then there exists a conjunct $X=x$ of $\vec{X} = \vec{x}$ and a possibly empty conjunction $\vec{Y}= \vec{y}$ such that $X=x \wedge \vec{Y} = \vec{y}$ is a cause of $\varphi$ in $\langle M, \vec{u}\rangle$.

    B. $\langle M, \vec{u}\rangle\models[\vec{X} = \vec{x}]\varphi$ for any $\langle M, \vec{u}\rangle \in K$.

2. $\vec{X}$ is a minimal set satisfying 1. 

3. $\vec{X} = \vec{x} \wedge \varphi$ hold in at least one member of $K$.

Moreover, an explanation is **non-trivial** just in case $\vec{X} = \vec{x}$ fails in at least one member of $K$.

## Implementation

First, we re-use the actual causality implementation discussed in another notebook. TODO add link

In [1]:
import functools

import numpy as np

import torch
from typing import Dict, List, Optional,  Union, Callable

import pandas as pd

import pyro
import pyro.distributions as dist

import random


from causal_pyro.indexed.ops import IndexSet, gather, indices_of, scatter
from causal_pyro.interventional.handlers import do
from causal_pyro.counterfactual.handlers import MultiWorldCounterfactual, Preemptions

In [2]:
class HalpernPearlModifiedApproximate:

    def __init__(
        self, 
        model: Callable,
        antecedents: Union[Dict[str, torch.Tensor], List[str]],
        outcome: str,
        witness_candidates: List[str],
        observations: Optional[Dict[str, torch.Tensor]],
        sample_size: int = 100,
        event_dim: int = 0
        ):
        
        self.model = model
        self.antecedents = antecedents
        self.outcome = outcome
        self.witness_candidates = witness_candidates
        self.observations = observations
        self.sample_size = sample_size

        self.antecedents_dict = (
            self.antecedents if isinstance(self.antecedents, dict)
            else self.revert_antecedents(self.antecedents)
        )
    
        self.preemptions = {candidate: functools.partial(self.preempt_with_factual,
                                             antecedents = self.antecedents) for 
                                             candidate in self.witness_candidates}
        

    @staticmethod
    def revert_antecedents(antecedents: List[str]) -> Dict[str, Callable[[torch.Tensor], torch.Tensor]]:
        return {antecedent: (lambda v: 1 - v) for antecedent in antecedents}

    @staticmethod   
    def preempt_with_factual(value: torch.Tensor, *,
                          antecedents: List[str] = None, event_dim: int = 0):
    
        if antecedents is None:
            antecedents = []

        antecedents = [a for a in antecedents if a in indices_of(value, event_dim=event_dim)]

        factual_value = gather(value, IndexSet(**{antecedent: {0} for antecedent in antecedents}),
                                event_dim=event_dim)
            
        return scatter({
            IndexSet(**{antecedent: {0} for antecedent in antecedents}): factual_value,
            IndexSet(**{antecedent: {1} for antecedent in antecedents}): factual_value,
        }, event_dim=event_dim)
        
        
    def __call__(self, *args, **kwargs):
        with pyro.poutine.trace() as trace:
            with MultiWorldCounterfactual():
                with do(actions=self.antecedents_dict):
                    with Preemptions(actions = self.preemptions):
                        with pyro.condition(data={k: torch.as_tensor(v) for k, v in self.observations.items()}):
                            with pyro.plate("plate", self.sample_size):
                                self.consequent = self.model()[self.outcome]
                                self.intervened_consequent = gather(self.consequent, IndexSet(**{ant: {1} for ant in self.antecedents}))
                                self.observed_consequent = gather(self.consequent, IndexSet(**{ant: {0} for ant in self.antecedents}))
                                self.consequent_differs = self.intervened_consequent != self.observed_consequent   
                                pyro.factor("consequent_differs", torch.where(self.consequent_differs, torch.tensor(0.0), torch.tensor(-1e8)))
                            
        self.trace = trace.trace

        # slightly hacky solution for odd witness candidate sets
        if  isinstance(self.consequent_differs.squeeze().tolist(), bool):
            self.existential_but_for = self.consequent_differs.squeeze()
        else:
            #if (len(self.consequent_differs.squeeze().tolist() )>1):
            self.existential_but_for = any(self.consequent_differs.squeeze().tolist()                )  

            

        witness_dict = dict()
        if self.witness_candidates:
            witness_keys = ["__split_" + candidate for candidate in self.witness_candidates]
            witness_dict = {key: self.trace.nodes[key]['value']  for key in witness_keys}
            
        witness_dict['observed'] = self.observed_consequent.squeeze()
        witness_dict['intervened'] = self.intervened_consequent.squeeze()
        witness_dict['consequent_differs'] = self.consequent_differs.squeeze()

        # slightly hacky as above
        self.witness_df = pd.DataFrame(witness_dict) if self.witness_candidates else witness_dict

    


Now for explanation. All the existential conditions present in the definition get us involved in sampling-based approximate searches for witnesses for existential claims. As there are a few clauses, we divide the labor.`factivity_check` will be used to test if a formula holds in a model. `part_of_minimal_cause` checks if a dictionary represents a conjunct in a minimal actual cause. `overlap_with_cause` deals with the conjunction business present in the definition. `ensurer` will test for condition 1B from the definition. `sufficient_cause` puts this together to test condition 1 of the definition. `explanation_check` puts these together paying attention to minimality requirements.

In [3]:
class HalpernPearlExplanationApproximate:

        def __init__(
            self,
            model: Callable,
            exogenous_variables: List[str],
            antecedents_dict: Dict[str, torch.Tensor],
            outcome_dict: Dict[str, torch.Tensor],
            nodes: List[str],
            excluded_settings: List[List[int]],
            runs_n: int = 100,
        ):

                self.model = model
                self.exogenous_variables = exogenous_variables
                self.antecedents_dict = antecedents_dict
                self.outcome_dict = outcome_dict
                self.nodes = nodes
                self.excluded_settings = excluded_settings
                self.runs_n = runs_n

        def factivity_check(self, model: Callable,
                             antecedents_dict: Dict[str, torch.Tensor],
                             outcome_dict: Dict[str, torch.Tensor],
                             observations: Dict[str, torch.Tensor]):

                with pyro.condition(data={k: torch.as_tensor(v) for k, v in observations.items()}):
                        output = model()
                        factivity_tensors = {k: torch.as_tensor(v) for k, v in list(antecedents_dict.items()) + list(outcome_dict.items())}
                        return all([factivity_tensors[key] == output[key] for key in factivity_tensors.keys()])


        def part_of_minimal_cause(self,
                                model: Callable,
                                antecedents: List[str],
                                outcome: str, 
                                nodes: List[str],
                                observations: Dict[str, torch.Tensor],
                                runs_n: int):

                cache = []
                minimal_antecedents = []

                for _ in range(1,runs_n):
                        if outcome in nodes:
                                nodes.remove(outcome)

                        companion_size = random.randint(0,len(nodes))
                        companion_candidates = random.sample(nodes, companion_size)

                        if set(companion_candidates) in cache:
                                continue
                
                        cache.append(set(companion_candidates))

                        witness_candidates = [node for node in nodes if 
                                        node not in antecedents and 
                                        node != outcome and 
                                        node not in companion_candidates]
                
                        HPM = HalpernPearlModifiedApproximate(
                                model = model,
                                antecedents = companion_candidates,
                                outcome =  outcome,
                                witness_candidates = witness_candidates,
                                observations = observations,
                                sample_size = 1000)
                        
                        HPM()

                        if  not HPM.existential_but_for:
                                continue
                
                        subset_is_a_minimal_cause = any([s.issubset(set(HPM.antecedents)) for s in minimal_antecedents])
                
                        if subset_is_a_minimal_cause:
                                continue
                        minimal_antecedents.append(set(HPM.antecedents))

                        for s in minimal_antecedents:
                                if set(HPM.antecedents).issubset(s) and s != set(HPM.antecedents):
                                        minimal_antecedents.remove(s)  


                return {"sufficient_cause": any([set(antecedents).issubset(s) for s in minimal_antecedents]),
                        "actual_cause": set(antecedents) in minimal_antecedents,
                        "minimal_antecedents" : minimal_antecedents, "cache": cache}

        
        def overlap_with_cause(self, model: Callable,
                                antecedents: List[str],
                                outcome: List[str],
                                nodes: List[str],
                                observations: Dict[str, torch.Tensor],
                                runs_n: int = 20):
    
                minimal_ante = self.part_of_minimal_cause(model, antecedents, outcome, nodes, 
                                                                  observations, runs_n)['minimal_antecedents']
                antecedents_set = set(antecedents)
    
                overlaps = [antecedents_set.intersection(s) for s in minimal_ante if antecedents_set.intersection(s)]
                overlap = any(overlaps)
                return {"overlap": overlap, "overlaps": overlaps    }


        def ensurer(self, model: Callable,
                exogenous_variables: List[str],
                antecedents_dict: Dict[str, torch.Tensor],
                outcome_dict: Dict[str, torch.Tensor], 
                runs_n: int):

                settings_cache = []
                intervened_consequent = []

                outcome = list(outcome_dict.keys())[0]
                antecedents = [key for key in antecedents_dict.keys()]

                for step in range(1,runs_n):
                        
                        random_setting = [random.choice([0., 1.]) for _ in range(len(exogenous_variables))]
                        
                        if random_setting in settings_cache:
                                continue
                        
                        settings_cache.append(random_setting)

                        observations = {var: val for var, val in zip(exogenous_variables, random_setting)}

                        with pyro.condition(data={k: torch.as_tensor(v) for k, v in observations.items()}):
                                with MultiWorldCounterfactual():
                                        with do(actions=antecedents_dict):
                                                intervened_consequent.append(
                                                gather(model()[outcome], 
                                                        IndexSet(**{ant: {1} for ant in antecedents})).squeeze().item())
                                        
                return {"ensurer": all(intervened_consequent),
                        "settings_cache": settings_cache, 
                        "intervened_consequent": intervened_consequent}
        

        
        def sufficient_cause(self, model, exogenous_variables, antecedents_dict, outcome_dict, nodes, observations, runs_n):

                factivity = self.factivity_check(model = model,
                                antecedents_dict = antecedents_dict,
                                outcome_dict = outcome_dict, 
                                observations = observations)
                
                if not factivity:
                        return {"sufficient_cause": False, "failure_reason": {"factivity": False}}   

                
                ensure = self.ensurer(model = model,
                        exogenous_variables = exogenous_variables,
                        antecedents_dict = antecedents_dict,
                        outcome_dict = outcome_dict,
                        runs_n = runs_n)['ensurer']
                
                if not ensure:
                        return {"sufficient_cause": False, "failure_reason": {"ensure": False}}
                
                overlap = self.overlap_with_cause(model = model,
                                        antecedents = [key for key in antecedents_dict.keys()],
                                        outcome =  list(outcome_dict.keys())[0],
                                        nodes =  nodes,
                                        observations = observations,
                                        runs_n= runs_n)
                
                if not overlap['overlap']:
                        return {"sufficient_cause": False, "failure_reason": {"overlap": False}}


                # minimality check starts here
                antecedents = [key for key in antecedents_dict.keys()]
                subsets = [[]]
                for node in antecedents:
                        subsets.extend([subset + [node] for subset in subsets])
                subsets.pop()

                
                for subset in subsets:
                
                        subset_ensure = self.ensurer(model = model,
                        exogenous_variables = exogenous_variables,
                        antecedents_dict = {key: antecedents_dict[key] for key in subset},
                        outcome_dict = outcome_dict,
                        runs_n = runs_n)['ensurer']    
                
                        if not subset_ensure:
                                continue

                        subset_overlap = self.overlap_with_cause(model = model,
                                antecedents = subset,
                                outcome =  list(outcome_dict.keys())[0],
                                nodes =  nodes,
                                observations = observations,
                                runs_n= runs_n)['overlap']

                        if subset_ensure and subset_overlap:
                        
                                return {"sufficient_cause": False, "failure_reason": {"minimality": False, "subset": subset}}
                # minimality check ends here 

                return {"sufficient_cause": True, "failure_reason": None}

        def explanation_check(self, model, exogenous_variables, antecedents_dict, outcome_dict, nodes, excluded_settings, runs_n):

                settings_cache = []
                factive_settings = []
                sufficient_causality_status = []
                failure_reasons = []
                possibility = False
                nontriviality = False

                outcome = list(outcome_dict.keys())[0]
                if outcome in nodes:
                        nodes.remove(outcome)

                for step in range(1,runs_n):
                        random_setting = [random.choice([0., 1.]) for _ in range(len(exogenous_variables))]

                        if random_setting in settings_cache or random_setting in excluded_settings:
                                continue

                        settings_cache.append(random_setting)
                        observations = {var: val for var, val in zip(exogenous_variables, random_setting)}

         
                        consequent_satisfied = self.factivity_check(model, {}, outcome_dict, observations)

                        if not consequent_satisfied:
                                continue
                
                        antecedent_satisfied = self.factivity_check(model, antecedents_dict, {}, observations)
            
                        if not antecedent_satisfied:
                                nontriviality = True

                        else:
            
                                possibility = True
                                factive_settings.append(random_setting)

                                sc = self.sufficient_cause(model = model,
                                                        exogenous_variables= exogenous_variables,
                                                        antecedents_dict = antecedents_dict,
                                                        outcome_dict = outcome_dict,
                                                        nodes = nodes,
                                                        observations= observations,
                                                        runs_n = 20)
                                
                                sufficient_causality_status.append(sc['sufficient_cause'])
                                failure_reasons.append(sc['failure_reason'])

                        if factive_settings:
                                explanation =  all(sufficient_causality_status)
                        else:
                                explanation = False

                return {"explanation": explanation,
                        "factive_settings": factive_settings,
                        "sufficient_causality_status": sufficient_causality_status,
                        "failure_reasons": failure_reasons,
                        "possibility": possibility,
                        "nontriviality": nontriviality,
                        "settings_cache": settings_cache}   


        def __call__(self, *args, **kwargs):

                self.explanation = self.explanation_check(
                        model = self.model,
                        exogenous_variables = self.exogenous_variables,
                        antecedents_dict = self.antecedents_dict,
                        outcome_dict = self.outcome_dict,
                        nodes = self.nodes,
                        excluded_settings= self.excluded_settings,
                        runs_n = self.runs_n)
                
                return self.model(*args, **kwargs)


## Examples

### Comments on example selection

- **forest fire:** one of the running examples in the book: we choose it, as it is the simplest model rich enough to illustrate a few interesting properties of explanation

- **extended forest fire:** the most complicated example discussed in the book, we use it to illustrate how the implementation handles the list and evaluation of potential explanations

### Forest fire

We've been using this example in previous notebooks. In this simplified model, a forest fire was caused by lightning or an arsonist, so we use three endogenous variables, and two exogenous variables corresponding to the two factors. In the conjunctive model, both factors have to be present for the fire to start. In the disjunctive model, each of them alone is sufficient. 

In [4]:
def ff_conjunctive():
    u_match_dropped = pyro.sample("u_match_dropped", dist.Bernoulli(0.5))
    u_lightning = pyro.sample("u_lightning", dist.Bernoulli(0.5))

    match_dropped = pyro.deterministic("match_dropped",
                                       u_match_dropped, event_dim=0)
    lightning = pyro.deterministic("lightning", u_lightning, event_dim=0)
    forest_fire = pyro.deterministic("forest_fire", torch.logical_and(match_dropped, lightning), event_dim=0).float()

    return {"match_dropped": match_dropped, "lightning": lightning,
            "forest_fire": forest_fire}

def ff_disjunctive():
    u_match_dropped = pyro.sample("u_match_dropped", dist.Bernoulli(0.5))
    u_lightning = pyro.sample("u_lightning", dist.Bernoulli(0.5))

    match_dropped = pyro.deterministic("match_dropped",
                                       u_match_dropped, event_dim=0)
    lightning = pyro.deterministic("lightning", u_lightning, event_dim=0)
    forest_fire = pyro.deterministic("forest_fire", torch.logical_or(match_dropped, lightning), event_dim=0).float()

    return {"match_dropped": match_dropped, "lightning": lightning,
            "forest_fire": forest_fire}


In [5]:
# Example 7.1.2. from the book

# all contexts available, no settings excluded by what 
# the agent knows about the world
# in the conjunctive model, the joint nodes are an explanation of forest fire

conjunctiveHPE = HalpernPearlExplanationApproximate(model = ff_conjunctive,
            exogenous_variables= ["u_match_dropped", "u_lightning"],
            antecedents_dict = {"match_dropped": 1., "lightning": 1.},
            outcome_dict = {"forest_fire": 1.},
            nodes= ["match_dropped", "lightning"],
            excluded_settings= [],
            runs_n = 20)

conjunctiveHPE()

conjunctiveHPE.explanation

{'explanation': True,
 'factive_settings': [[1.0, 1.0]],
 'sufficient_causality_status': [True],
 'failure_reasons': [None],
 'possibility': True,
 'nontriviality': False,
 'settings_cache': [[0.0, 1.0], [1.0, 1.0], [0.0, 0.0], [1.0, 0.0]]}

In [6]:
# But none individually is (see e.g. match_dropped)

conjunctive_split_HPE = HalpernPearlExplanationApproximate(model = ff_conjunctive,
            exogenous_variables= ["u_match_dropped", "u_lightning"],
            antecedents_dict = {"match_dropped": 1.},
            outcome_dict = {"forest_fire": 1.},
            nodes= ["match_dropped", "lightning"],
            excluded_settings= [],
            runs_n = 20)

conjunctive_split_HPE()

conjunctive_split_HPE.explanation

{'explanation': False,
 'factive_settings': [[1.0, 1.0]],
 'sufficient_causality_status': [False],
 'failure_reasons': [{'ensure': False}],
 'possibility': True,
 'nontriviality': False,
 'settings_cache': [[0.0, 1.0], [1.0, 0.0], [0.0, 0.0], [1.0, 1.0]]}

In [7]:
# The situation is the opposite for the disjunctive model
# the joint nodes are not an explanation of ff
# each of the invidual nodes is an explanation of ff

disjunctiveHPE = HalpernPearlExplanationApproximate(model = ff_disjunctive,
            exogenous_variables= ["u_match_dropped", "u_lightning"],
            antecedents_dict = {"match_dropped": 1., "lightning": 1.},
            outcome_dict = {"forest_fire": 1.},
            nodes= ["match_dropped", "lightning"],
            excluded_settings= [],
            runs_n = 20)

disjunctiveHPE()

disjunctiveHPE.explanation

{'explanation': False,
 'factive_settings': [[1.0, 1.0]],
 'sufficient_causality_status': [False],
 'failure_reasons': [{'minimality': False, 'subset': ['match_dropped']}],
 'possibility': True,
 'nontriviality': True,
 'settings_cache': [[0.0, 1.0], [1.0, 0.0], [1.0, 1.0], [0.0, 0.0]]}

In [8]:
disjunctive_split_HPE = HalpernPearlExplanationApproximate(model = ff_disjunctive,
            exogenous_variables= ["u_match_dropped", "u_lightning"],
            antecedents_dict = {"match_dropped": 1.},
            outcome_dict = {"forest_fire": 1.},
            nodes= ["match_dropped", "lightning"],
            excluded_settings= [],
            runs_n = 20)

disjunctive_split_HPE()

disjunctive_split_HPE.explanation

{'explanation': True,
 'factive_settings': [[1.0, 0.0], [1.0, 1.0]],
 'sufficient_causality_status': [True, True],
 'failure_reasons': [None, None],
 'possibility': True,
 'nontriviality': True,
 'settings_cache': [[0.0, 1.0], [1.0, 0.0], [0.0, 0.0], [1.0, 1.0]]}

In [10]:
# in a setting where we exclude  match and no lightning
# triviality ensues, as expected

disjunctive_excluded_HPE = HalpernPearlExplanationApproximate(model = ff_disjunctive,
            exogenous_variables= ["u_match_dropped", "u_lightning"],
            antecedents_dict = {"lightning": 1.},
            outcome_dict = {"forest_fire": 1.},
            nodes= ["match_dropped"],
            excluded_settings= [[1.0, 0.0]],
            runs_n = 20)

disjunctive_excluded_HPE()

disjunctive_excluded_HPE.explanation

{'explanation': False,
 'factive_settings': [[0.0, 1.0], [1.0, 1.0]],
 'sufficient_causality_status': [False, False],
 'failure_reasons': [{'overlap': False}, {'overlap': False}],
 'possibility': True,
 'nontriviality': False,
 'settings_cache': [[0.0, 1.0], [1.0, 1.0], [0.0, 0.0]]}

### Extended forest fire

In April, given the electrical storm in May, the forest would have caught fire in May (and not in June). However, given the storm, if there had been an electrical storm only in May, the forest
would not have caught fire at all; if there had been an electrical storm only in June, it would have caught fire in June. The model has five endogenous variables: `ar` for *April rains*, 
`esm` for *electric storms in May*, `esj` for *electric storms in June*, `ffm` for *forest fire in May*, `ffj` for *forest fire in June* and `ff` for *forest fire either in May or in June (or both)*. 

In [12]:
def ff_extended():
    u_ar = pyro.sample("u_ar", dist.Bernoulli(0.5))
    u_esm = pyro.sample("u_esm", dist.Bernoulli(0.5))
    u_esj = pyro.sample("u_esj", dist.Bernoulli(0.5))

    ar = pyro.deterministic("ar", u_ar, event_dim=0)
    esm = pyro.deterministic("esm", u_esm, event_dim=0)
    esj = pyro.deterministic("esj", u_esj, event_dim=0)

    ffm = pyro.deterministic("ffm", torch.logical_and(esm, ~ ar.bool()), event_dim=0).float()
    ffj = pyro.deterministic("ffj", torch.logical_and(esj, (ar.bool() | ~ esm.bool())), event_dim=0).float()

    ff = pyro.deterministic("ff", torch.logical_or(ffm, ffj), event_dim=0).float()

    return {"u_ar": u_ar, "u_esm": u_esm, "u_esj": u_esj, 
            "ar": ar, "esm": esm, "esj": esj, "ffm": ffm, "ffj": ffj, "ff": ff}


In [13]:
# with no excluded settings
# one explanation for ff is esj

ff_extended_esj_HPE = HalpernPearlExplanationApproximate(model = ff_extended,
            exogenous_variables= ["u_ar", "u_esm", "u_esj"],
            antecedents_dict = {"esj": 1.},
            outcome_dict = {"ff": 1.},
            nodes= ["ar", "esm", "esj"],
            excluded_settings= [],
            runs_n = 20)

ff_extended_esj_HPE()
ff_extended_esj_HPE.explanation

{'explanation': True,
 'factive_settings': [[1.0, 0.0, 1.0],
  [0.0, 1.0, 1.0],
  [1.0, 1.0, 1.0],
  [0.0, 0.0, 1.0]],
 'sufficient_causality_status': [True, True, True, True],
 'failure_reasons': [None, None, None, None],
 'possibility': True,
 'nontriviality': True,
 'settings_cache': [[0.0, 1.0, 0.0],
  [1.0, 0.0, 0.0],
  [1.0, 0.0, 1.0],
  [0.0, 1.0, 1.0],
  [0.0, 0.0, 0.0],
  [1.0, 1.0, 1.0],
  [0.0, 0.0, 1.0]]}

In [14]:
# another involves esm = 1 and ar = 0 
ff_extended_esmar_HPE = HalpernPearlExplanationApproximate(model = ff_extended,
            exogenous_variables= ["u_ar", "u_esm", "u_esj"],
            antecedents_dict = {"esm": 1., "ar": 0.},
            outcome_dict = {"ff": 1.},
            nodes= ["ar", "esm", "esj"],
            excluded_settings= [],
            runs_n = 20)

ff_extended_esmar_HPE()
ff_extended_esmar_HPE.explanation

{'explanation': True,
 'factive_settings': [[0.0, 1.0, 1.0], [0.0, 1.0, 0.0]],
 'sufficient_causality_status': [True, True],
 'failure_reasons': [None, None],
 'possibility': True,
 'nontriviality': True,
 'settings_cache': [[0.0, 0.0, 0.0],
  [1.0, 0.0, 0.0],
  [0.0, 1.0, 1.0],
  [0.0, 1.0, 0.0],
  [1.0, 1.0, 0.0],
  [1.0, 0.0, 1.0]]}

In [15]:
# explore all 27 possible explanations
# to confirm that these are the only 
# possible explanations

explanatory_status = []
candidates = []

nodes = ["ar", "esm", "esj"]
subsets = [[]]
for node in nodes:
        subsets.extend([subset + [node] for subset in subsets])


for subset in subsets:
        for _ in range(50):
                random_setting = [random.choice([0., 1.]) 
                          for _ in range(len(subset))]
                candidate = {var: val for var, val in zip(subset, random_setting)}

                if candidate in candidates:
                        continue

                candidates.append(candidate)

                ffHPE = HalpernPearlExplanationApproximate(model = ff_extended,
                        exogenous_variables= ["u_ar", "u_esm", "u_esj"],
                        antecedents_dict = candidate,
                        outcome_dict = {"ff": 1.},
                        nodes= ["ar", "esm", "esj"],
                        excluded_settings= [],
                        runs_n = 20)

                ffHPE()
                explanatory_status.append(ffHPE.explanation['explanation'])
        

explanation_search = pd.DataFrame({"candidates": candidates,
                                   "explanatory_status": explanatory_status})

print(explanation_search)


                             candidates  explanatory_status
0                                    {}               False
1                           {'ar': 1.0}               False
2                           {'ar': 0.0}               False
3                          {'esm': 0.0}               False
4                          {'esm': 1.0}               False
5               {'ar': 0.0, 'esm': 1.0}                True
6               {'ar': 1.0, 'esm': 0.0}               False
7               {'ar': 1.0, 'esm': 1.0}               False
8               {'ar': 0.0, 'esm': 0.0}               False
9                          {'esj': 1.0}                True
10                         {'esj': 0.0}               False
11              {'ar': 0.0, 'esj': 0.0}               False
12              {'ar': 0.0, 'esj': 1.0}               False
13              {'ar': 1.0, 'esj': 1.0}               False
14              {'ar': 1.0, 'esj': 0.0}               False
15             {'esm': 1.0, 'esj': 1.0} 